<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,cv2,scipy
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal
import numpy as np

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try:
        x = x.strip()
        if x[0] == '/' or x[0:2] == './':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                x = 'https://' + x
                x = requests.get(x, timeout=3)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                x = x.replace('github.com', 'raw.githubusercontent.com')
                mod = x.split('/')
                for s in ['/main/', '/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(x, timeout=3)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(x, timeout=3)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except Exception as err:
        sys.stdout.write(f'request {x} :{err}\n')

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


In [2]:
import cv2
import math
from scipy.stats import mode

In [3]:
# _IMPORT('https://gitee.com/qrsforever/nb_easy/easy_utils.py')
# _IMPORT('https://gitee.com/qrsforever/nb_easy/easy_task.py')
_IMPORT('/data/nb_easy/easy_utils.py')
_IMPORT('/data/nb_easy/easy_task.py')

In [4]:
logger = nbeasy_get_logger('pipeline', level='info', console=True, mp=True)

In [5]:
class VideoRemoveStillWorker(ICallable):
    def __init__(self, outdir, video_path, focus_box=None, black_box=None, rmstill=None, debug=False):
        self.video_path = video_path
        self.focus_box = focus_box
        self.black_box = black_box
        self.rmstill = rmstill  # bin_thres=20, area_rate_thres=0.01
        self.debug = debug
        self.outdir = outdir

    def init(self):
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            raise
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        r = cap.get(cv2.CAP_PROP_FPS)
        if self.focus_box:
            self.fx1, self.fy1 = int(w * self.focus_box[0]), int(h * self.focus_box[1])
            self.fx2, self.fy2 = int(w * self.focus_box[2]), int(h * self.focus_box[3])
        if self.black_box:
            self.bx1, self.by1 = int(w * self.black_box[0]), int(h * self.black_box[1])
            self.bx2, self.by1 = int(w * self.black_box[2]), int(h * self.black_box[3])
        self.cap = cap
        self.num = n
        self.fps = r
        self.frw = self.fx2 - self.fx1
        self.frh = self.fy2 - self.fy1
        if self.rmstill:
            self.area_thres = math.ceil(self.rmstill['area_rate_thres'] * self.frw * self.frh)
            self.bin_thres = self.rmstill['bin_thres']
            self.norm_brightness = self.rmstill['norm_brightness']
        if self.debug:
            self.keepidxes = []
            self.binpoints = []
            self.binframes = []

    def __call__(self, x, shm_frame_out):
        w, h = self.frw, self.frh
        frame_pre = None
        i = -1
        while True:
            success, frame_bgr = self.cap.read()
            if not success:
                break
            i += 1
            keep_flag = False
            if self.black_box:
                frame_bgr[self.by1:self.by2, self.bx1:self.bx2, :] = 0
            if self.focus_box:
                frame_bgr = frame_bgr[self.fy1:self.fy2, self.fx1:self.fx2, :]
            if self.rmstill:
                if self.norm_brightness:
                    h, s, v = cv2.split(cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV))
                    v = np.array((v - np.mean(v)) / np.std(v) * 32 + 127, dtype=np.uint8)
                    frame_bgr = cv2.cvtColor(cv2.merge([h, s, v]), cv2.COLOR_HSV2BGR)
                frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
                if frame_pre is not None:
                    frame_tmp = cv2.absdiff(frame_gray, frame_pre)
                    frame_tmp = cv2.threshold(frame_tmp, self.bin_thres, 255, cv2.THRESH_BINARY)[1]
                    frame_tmp = cv2.erode(frame_tmp, None, iterations=1)
                    frame_tmp = cv2.dilate(frame_tmp, None, iterations=1)
                    val = np.sum(frame_tmp == 255)
                    if val > self.area_thres:
                        keep_flag = True
                        if self.debug:
                            self.keepidxes.append(i)
                            self.binpoints.append(np.round(val / self.area_thres, 2))
                            self.binframes.append(cv2.resize(frame_tmp, (120, 120)))
                frame_pre = frame_gray
            
            if i % 500 == 0:
                logger.info('process frame progress: %.2f' % (i * 100 / self.num))
            
            if keep_flag:
                shm_frame_out.lock()
                shm_frame_out.data = frame_bgr
                yield {'frame_idx': i, 'progress': round(i / self.num, 2)}
        yield State.STOP

    def shutdown(self):
        if self.cap:
            self.cap.release()
        if self.debug:
            np.save(f'{self.outdir}/keepidxes.npy', self.keepidxes)
            np.save(f'{self.outdir}/binpoints.npy', self.binpoints)
            np.save(f'{self.outdir}/binframes.npy', np.asarray(self.binframes))


class VideoOpticalFlowWorker(ICallable):
    def __init__(self, outdir, angle_ranges=[], mag_thres=5, of_params={}, debug=False):
        self.params = {
            'pyr_scale': of_params.get('pyr_scale', 0.5),
            'levels': of_params.get('levels', 3),
            'winsize': of_params.get('winsize', 15),
            'iterations': of_params.get('iterations', 3),
            'poly_n': of_params.get('poly_n', 5),
            'poly_sigma': of_params.get('poly_sigma', 1.1),
            'flags': cv2.OPTFLOW_LK_GET_MIN_EIGENVALS
        }
        self.angle_ranges = angle_ranges
        self.mag_thres = mag_thres
        self.outdir = outdir
        self.debug = debug
    def init(self):
        self.frame_pre = None
        self.stat_count = 0

    def __call__(self, x, shm_frame_in, shm_frame_out):
        self.stat_count += 1
        frame_bgr = shm_frame_in.data.copy().astype(np.uint8)
        shm_frame_in.unlock()
        frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
        if self.frame_pre is None:
            self.frame_pre = frame_gray
        else:
            flow = cv2.calcOpticalFlowFarneback(self.frame_pre, frame_gray, None, **self.params)
            mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees=True)
            move_mode = mode(ang[mag > self.mag_thres])[0]
            keep_flag = False
            for r in self.angle_ranges:
                if r[0] < move_mode <= r[1]:
                    keep_flag = True
                    break
            if keep_flag:
                shm_frame_out.lock()
                shm_frame_out.data = frame_bgr
                yield x
 
    def shutdown(self):
        logger.info(f'optical flow call count: {self.stat_count}')


class VideoModelInputsWorker(ICallable):
    def __init__(self, outdir, input_width=112, input_height=112, rep_count=1, rot_angle=0, debug=False):
        self.input_width, self.input_height = input_width, input_height
        self.rep_count = rep_count
        self.rot_angle = rot_angle
        self.outdir = outdir
        self.debug = debug
        
    def init(self):
        if self.rot_angle > 0:
            width, height = self.input_width, self.input_height
            self.rot_matrix = cv2.getRotationMatrix2D(center=(int(width / 2), int(height / 2)), angle=self.rot_angle, scale=1.0)
        else:
            self.rot_matrix = None
        self.model_frames = []
        self.model_inputs = []
            
    def __call__(self, x, shm_frame_in):
        width, height = self.input_width, self.input_height
        rep_count, rot_matrix = self.rep_count, self.rot_matrix
        frame_bgr = shm_frame_in.data.copy().astype(np.uint8)
        shm_frame_in.unlock()
        if self.rep_count > 1:
            frame_bgr = np.hstack([frame_bgr] * self.rep_count)
            frame_bgr = np.vstack([frame_bgr] * self.rep_count)
        frame_bgr = cv2.resize(frame_bgr, (width, height))
        if rot_matrix is not None:
            frame_bgr = cv2.warpAffine(frame_bgr, rot_matrix, (width, height))
        
        self.model_frames.append(x['frame_idx'])
        self.model_inputs.append(frame_bgr)

    def shutdown(self):
        logger.info(f'model input call count: {len(self.model_frames)}')
        np.save(f'{self.outdir}/model_frames.npy', self.model_frames)
        np.save(f'{self.outdir}/model_inputs.npy', np.asarray(self.model_inputs))
        # if len(self.model_inputs) > 0:
        #     w, h, c = self.model_inputs[0].shape
        #     mp4_file = os.path.join('/data', 'test.mp4')
        #     mp4_vid = cv2.VideoWriter(mp4_file, cv2.VideoWriter_fourcc(*'mp4v'), 25.0, (w, h))
        #     for frame in self.model_inputs:
        #         mp4_vid.write(frame)
        #     mp4_vid.release()

In [8]:
import tempfile

focus_box=[0.318, 0.052, 0.986, 1]

rmstill = {
    'bin_thres': 20,
    'area_rate_thres': 0.03,
    'norm_brightness': False,
}

of_params = {
    'pyr_scale': 0.5,
    'levels': 3,
    'winsize': 15,
    'iterations': 3,
    'poly_n': 5,
    'poly_sigma': 1.1,
    'flags': cv2.OPTFLOW_LK_GET_MIN_EIGENVALS
}

angle_ranges = [(135, 225)]

video_path = '/data/20211209080518.mp4'

with tempfile.TemporaryDirectory() as out_dir:
    logger.info(out_dir)
    pipeline = TaskPipelineV2()
    pipeline.add(VideoRemoveStillWorker(out_dir, video_path, focus_box, rmstill=rmstill, debug=True), shms=[ShmNumpy('I', (640, 352, 3))])
    # pipeline.add(VideoOpticalFlowWorker(out_dir, angle_ranges, 5, of_params=of_params, debug=True), shms=[ShmNumpy('I', (640, 352, 3))])
    pipeline.add(VideoModelInputsWorker(out_dir, 112, 112, debug=True))
    pipeline.run()

2022-02-18 17:39:43,059 - <module>:26 - pipeline - INFO - /tmp/tmps0hzlfej
2022-02-18 17:39:43,092 - __call__:73 - pipeline - INFO - process frame progress: 0.00
2022-02-18 17:39:43,577 - __call__:73 - pipeline - INFO - process frame progress: 6.58
2022-02-18 17:39:44,046 - __call__:73 - pipeline - INFO - process frame progress: 13.16
2022-02-18 17:39:44,613 - __call__:73 - pipeline - INFO - process frame progress: 19.74
2022-02-18 17:39:45,069 - __call__:73 - pipeline - INFO - process frame progress: 26.32
2022-02-18 17:39:45,515 - __call__:73 - pipeline - INFO - process frame progress: 32.89
2022-02-18 17:39:45,969 - __call__:73 - pipeline - INFO - process frame progress: 39.47
2022-02-18 17:39:46,426 - __call__:73 - pipeline - INFO - process frame progress: 46.05
2022-02-18 17:39:46,871 - __call__:73 - pipeline - INFO - process frame progress: 52.63
2022-02-18 17:39:47,318 - __call__:73 - pipeline - INFO - process frame progress: 59.21
2022-02-18 17:39:47,769 - __call__:73 - pipelin